# SQL to pandas cheat sheet

This is a cheat sheet for the comparison between SQL commands to python's library Pandas.

In [1]:
# Import libraries
import pandas as pd
import numpy as np

# Install pydataset
#!pip install pydataset

# Import data from pydataset library (used as example data)
from pydataset import data

In [2]:
# Load iris dataset 
df = data('iris')
df.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa


## SQL WHERE Clause in pandas

Used to extract only those records that fulfill a specified condition. Similar to the SQL command:

<code>SELECT *
FROM table_name
WHERE condition</code>

In [3]:
df[df['Species']=='setosa'].head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,5.1,3.5,1.4,0.2,setosa
2,4.9,3.0,1.4,0.2,setosa
3,4.7,3.2,1.3,0.2,setosa
4,4.6,3.1,1.5,0.2,setosa
5,5.0,3.6,1.4,0.2,setosa


To select some columns like:
    
<code>SELECT column1, column2, ...
FROM table_name
WHERE condition</code>

In [17]:
df[df['Species']=='setosa'].loc[:, ['Sepal.Length', 'Sepal.Width']].head()

,Sepal.Length,Sepal.Width
1,5.1,3.5
2,4.9,3.0
3,4.7,3.2
4,4.6,3.1
5,5.0,3.6


## SQL Distinct in pandas

Similar to the SQL command <code> SELECT DISTINCT COL_1 FROM TABLE </code>

### Method 1

In [4]:
df['Species'].unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

In [5]:
df.Species.unique()

array(['setosa', 'versicolor', 'virginica'], dtype=object)

## SQL Order by by in pandas

Similar to the SQL command:

<code>SELECT column1, column2, ...
FROM table_name
ORDER BY column1 ASC, column2 DESC, ... 
</code>  

In [11]:
df.sort_values(by = ['Sepal.Length', 'Petal.Length'], axis = 0, ascending = [False, True]).head(5)

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
132,7.9,3.8,6.4,2.0,virginica
136,7.7,3.0,6.1,2.3,virginica
118,7.7,3.8,6.7,2.2,virginica
123,7.7,2.8,6.7,2.0,virginica
119,7.7,2.6,6.9,2.3,virginica


## SQL Group by in pandas

### Group by Count

Similar to the SQL command
<code> SELECT COL_1, COUNT(*) FROM TABLE GROUP BY COL_1 </code>

In [18]:
df.groupby(['Species']).size()

Species
setosa        50
versicolor    50
virginica     50
dtype: int64

### Group by and Aggregate by different functions

GROUP BY with different aggregate functions (COUNT, MAX, MIN, SUM, AVG) to group the result-set by one or more columns. Similar to the SQL command:

<code>SELECT COUNT(COL_1), SUM(COL_2)
       FROM TABLE
       GROUP BY COL_3, COL_4
</code>   
 

In [29]:
df.groupby('Species').agg({'Sepal.Length':['max', 'min'], 
                         'Sepal.Width':'mean', 
                         'Petal.Length':'sum', 
                         'Petal.Width': lambda x: x.max() - x.min()})

Sepal.Length      Sepal.Width Petal.Length Petal.Width
                    max  min        mean          sum    <lambda>
Species                                                          
setosa              5.8  4.3       3.428         73.1         0.5
versicolor          7.0  4.9       2.770        213.0         0.8
virginica           7.9  4.9       2.974        277.6         1.1

In [23]:
df.groupby('Species').agg({'Sepal.Length':['max', 'min'], 
                         'Sepal.Width':'mean', 
                         'Petal.Length':'sum', 
                         'Petal.Width': [('Max minus min', lambda x: x.max() - x.min())]})

Sepal.Length      Sepal.Width Petal.Length   Petal.Width
                    max  min        mean          sum Max minus min
Species                                                            
setosa              5.8  4.3       3.428         73.1           0.5
versicolor          7.0  4.9       2.770        213.0           0.8
virginica           7.9  4.9       2.974        277.6           1.1

Another form to avoid the "lambda" column name, is to create a function and supply a custom name

In [44]:
def max_min(x):
    return x.max() - x.min()

max_min.__name__ = 'Max minus Min'

df.groupby('Species').agg({'Sepal.Length':['max', 'min'], 
                         'Sepal.Width':'mean', 
                         'Petal.Length':'sum', 
                         'Petal.Width': max_min})

Sepal.Length      Sepal.Width Petal.Length   Petal.Width
                    max  min        mean          sum Max minus Min
Species                                                            
setosa              5.8  4.3       3.428         73.1           0.5
versicolor          7.0  4.9       2.770        213.0           0.8
virginica           7.9  4.9       2.974        277.6           1.1

To flatten the hierarchical index in columns

In [45]:
df_groups = df.groupby('Species').agg({'Sepal.Length':['max', 'min'], 
                         'Sepal.Width':'mean', 
                         'Petal.Length':'sum', 
                         'Petal.Width': max_min})

df_groups.columns

MultiIndex([('Sepal.Length',           'max'),
            ('Sepal.Length',           'min'),
            ( 'Sepal.Width',          'mean'),
            ('Petal.Length',           'sum'),
            ( 'Petal.Width', 'Max minus Min')],
           )

In [47]:
df_groups.columns = [' '.join(col).strip() for col in df_1.columns.values]
df_groups

,Sepal.Length max,Sepal.Length min,Sepal.Width mean,Petal.Length sum,Petal.Width Max minus Min
Species,,,,,
setosa,5.8,4.3,3.428,73.1,0.5
versicolor,7.0,4.9,2.770,213.0,0.8
virginica,7.9,4.9,2.974,277.6,1.1


### Group by with conditional count

Similar to the SQL command:

<code>SELECT COL_1, SUM(CASE WHEN condition1 THEN 1 ELSE 0)
        FROM TABLE
        GROUP BY COL_1
</code>   
 

In [15]:
df.groupby('Species')['Sepal.Length'].apply(lambda x: (x>6).sum())

Species
setosa         0
versicolor    20
virginica     41
Name: Sepal.Length, dtype: int64

## SQL CASE Statement in pandas

Used for creating new columns based on multiple conditions. Similar to the SQL command:

<code>CASE
    WHEN condition1 THEN result1
    WHEN condition2 THEN result2
    WHEN conditionN THEN resultN
    ELSE result
END </code>

In [19]:
# Define a function with the rules to apply

def create_column(df):
    
    if df['Species'] == 'setosa':
        return 1
    elif df['Species'] == 'versicolor':
        return 2
    elif df['Species'] == 'virginica':
        return 3
    
# Then create the column and apply the create_column function

df['Cod.Species'] = df.apply(create_column, axis = 1)
df.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species,Cod.Species
1,5.1,3.5,1.4,0.2,setosa,1
2,4.9,3.0,1.4,0.2,setosa,1
3,4.7,3.2,1.3,0.2,setosa,1
4,4.6,3.1,1.5,0.2,setosa,1
5,5.0,3.6,1.4,0.2,setosa,1


##  SQL UPDATE Statement in pandas

Similar to the SQL command:

<code>UPDATE table_name
SET column1 = value1, column2 = value2, ...
WHERE condition;
</code>   

Let's update the 'Species' columns by capitalizing the first letter.

In [17]:
df['Species'] = df['Species'].apply(lambda x: x.capitalize())
df.head()

,Sepal.Length,Sepal.Width,Petal.Length,Petal.Width,Species
1,5.1,3.5,1.4,0.2,Setosa
2,4.9,3.0,1.4,0.2,Setosa
3,4.7,3.2,1.3,0.2,Setosa
4,4.6,3.1,1.5,0.2,Setosa
5,5.0,3.6,1.4,0.2,Setosa


## SQL Joins in pandas

### SQL Left Join

Similar to the SQL command:

<code>SELECT column_name(s)
FROM table1
LEFT JOIN table2
ON table1.column_name = table2.column_name
</code>   

Let's create two fictitious data frames: crop_yield and climate_features

In [37]:
crop_yield = pd.DataFrame(
{'Year': [2019, 2020, 2020, 2020, 2020, 2019, 2020, 2019, 2020, 2019],
'Land Lot':['P', 'S', 'S', 'S', 'V', 'A', 'P', 'S', 'V', 'V'],
'Crop Yield': [86, 92, 94, 39, 136, 118, 81, 121, 34, 121]}
)
crop_yield.head(5)

,Year,Land Lot,Crop Yield
0,2019,P,86
1,2020,S,92
2,2020,S,94
3,2020,S,39
4,2020,V,136


In [38]:
climate_features = pd.DataFrame(
{'Year': [2019, 2019, 2019, 2019, 2019, 2020, 2020, 2020, 2020, 2020],
 'Land Lot': ['A', 'C', 'P', 'S', 'V', 'A', 'C', 'P', 'S', 'V'],
 'Longest dry spell': [2, 6, 1, 4, 1, 2, 5, 5, 3, 2],
 'Longest wet spell': [41, 41, 41, 38, 43, 45, 67, 48, 68, 47]}
)
climate_features.head(5)

,Year,Land Lot,Longest dry spell,Longest wet spell
0,2019,A,2,41
1,2019,C,6,41
2,2019,P,1,41
3,2019,S,4,38
4,2019,V,1,43


In [39]:
pd.merge(crop_yield,climate_features,on=['Year', 'Land Lot'], how='left')

,Year,Land Lot,Crop Yield,Longest dry spell,Longest wet spell
0,2019,P,86,1,41
1,2020,S,92,3,68
2,2020,S,94,3,68
3,2020,S,39,3,68
4,2020,V,136,2,47
5,2019,A,118,2,41
6,2020,P,81,5,48
7,2019,S,121,4,38
8,2020,V,34,2,47
9,2019,V,121,1,43
